In [323]:
import torch.nn.functional as F
from torch import nn
import copy
import torch

class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class EncoderOnly(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, d_model, layer, N, dropout=0.1):
        super(EncoderOnly, self).__init__()
        self.pe = PositionalEncoding(d_model, dropout)
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        x = self.pe(x)
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)
    
def initialize(module):
    for p in module.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

def make_model(src_vocab, per_layer_embeddings, N=2, 
               d_model=128, d_ff=512, h=2, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
#     position = PositionalEncoding(d_model, dropout)

    enc = EncoderOnly(d_model, EncoderLayer(d_model, c(attn), c(ff), dropout), N, dropout)
    
    decoders = []
    for num_embs in per_layer_embeddings:
        decoders.append(nn.Sequential(*[
            Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N)
            , Generator(d_model, num_embs)
        ]))
#     dec = Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N)
#     gen = Generator(d_model, tgt_vocab)
#     components = [enc, dec, src_emb, tgt_emb, gen]
        
    for dec in decoders:
        initialize(dec)
        
    initialize(enc)
    
    return enc, decoders

In [324]:
options_in = 35
per_layer_options_out = [5, 4]
d_model = 64

In [325]:
enc, decoders = make_model(options_in, per_layer_options_out, d_model=d_model)

In [326]:
from ipdb import set_trace
import numpy as np

embeddings = torch.rand(options_in, d_model)

batch_of_embeddings = [embeddings.clone().unsqueeze(0) for _ in range(batch_size)]
batch_of_embeddings = torch.cat(batch_of_embeddings)

batch_of_masks = torch.tensor(np.random.randint(options_in, size=(batch_size, 1))).float()

for emb, mask in zip(batch_of_embeddings, batch_of_masks):
    emb[mask.int().item():] = 0

In [327]:
encode_step = enc(batch_of_embeddings)

tensor([[[-1.6065e+00, -1.5170e-01,  8.0579e-01,  ...,  5.6617e-01,
           7.5357e-01,  1.5010e-01],
         [-5.4105e-01,  1.9623e-01,  6.6845e-01,  ...,  1.3914e+00,
          -2.2179e-01,  8.7144e-01],
         [ 1.7966e-01, -1.3180e+00,  1.1345e+00,  ...,  1.4641e+00,
           8.8721e-01,  4.1921e-02],
         ...,
         [-1.0597e+00,  2.7805e-01,  6.2312e-02,  ...,  9.4419e-01,
           7.1273e-01,  2.3636e-01],
         [-2.9419e-01, -5.2450e-02,  2.5608e-01,  ...,  1.2792e+00,
           5.6235e-01,  3.1980e-02],
         [-8.7824e-01, -2.3800e-01,  7.5172e-01,  ...,  1.3273e+00,
           6.7777e-01,  8.7528e-01]],

        [[-1.4586e+00,  7.8804e-01, -7.9737e-01,  ...,  1.4474e+00,
          -6.0273e-01, -2.3895e-01],
         [-1.1197e+00, -1.2546e+00,  5.3635e-01,  ...,  1.3617e+00,
           5.2885e-01,  4.9019e-01],
         [-1.2900e-03, -1.0283e+00,  8.7424e-01,  ...,  1.8584e+00,
           7.3960e-02,  3.9833e-01],
         ...,
         [-7.7543e-01,  2

In [90]:
encode_step.shape
#

torch.Size([35, 35, 128])

In [37]:
# enc()

In [ ]:
 def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)